### Setting up S3 VPC Gateway Endpoint

Step 1 - Create a S3 bucket 

 `aws s3 mb s3://_______` where the blank space is the name of the bucket like ` aws s3 mb s3://semistify` 

Step 2 - S3 Gateway points

a. Identify the VPC that needs access to S3 using `aws ec2 describe-vpcs`

output like this:

In [ ]:
{   "Vpcs": [
        {
            "CidrBlock": "172.31.0.0/16",
            "DhcpOptionsId": "dopt-045e9b07b84816bff",
            "State": "available",
            "VpcId": "vpc-03cc1cc1e02d0160d",
            "OwnerId": "066642259352",
            "InstanceTenancy": "default",
            "CidrBlockAssociationSet": [
                {
                    "AssociationId": "vpc-cidr-assoc-0958012b90f676611",
                    "CidrBlock": "172.31.0.0/16",
                    "CidrBlockState": {
                        "State": "associated"
                    }
                }
            ],
            "IsDefault": true
        }
    ]
}

b. Identify the Routing table you want to configure with your VPC Gateway using `aws ec2 describe-route-tables`

Output like this:

In [ ]:
{
    "RouteTables": [
        {
            "Associations": [
                {
                    "Main": true,
                    "RouteTableAssociationId": "rtbassoc-00ba3298a85a29c61",
                    "RouteTableId": "rtb-0daedd7d2022f5ac6",
                    "AssociationState": {
                        "State": "associated"
                    }
                }
            ],
            "PropagatingVgws": [],
            "RouteTableId": "rtb-0daedd7d2022f5ac6",
            "Routes": [
                {
                    "DestinationCidrBlock": "172.31.0.0/16",
                    "GatewayId": "local",
                    "Origin": "CreateRouteTable",
                    "State": "active"
                },
                {
                    "DestinationCidrBlock": "0.0.0.0/0",
                    "GatewayId": "igw-09e6be4ae3236dada",
                    "Origin": "CreateRoute",
                    "State": "active"
                }
            ],
            "Tags": [],
            "VpcId": "vpc-03cc1cc1e02d0160d",
            "OwnerId": "066642259352"
        }
    ]
}

c. Create an S3 Gateway Endpoint with `aws ec2 create-vpc-endpoint --vpc-id _____ --service-name com.amazonaws.us-east-1.s3 --route-table-ids _________` replacing the blacns with the VPC and Routing Table Ids

`aws ec2 create-vpc-endpoint --vpc-id vpc-03cc1cc1e02d0160d --service-name com.amazonaws.us-east-1.s3 --route-table-ids rtb-0daedd7d2022f5ac6`

In [ ]:
}
    "VpcEndpoint": {
        "VpcEndpointId": "vpce-08390c67d9ee98e77",
        "VpcEndpointType": "Gateway",
        "VpcId": "vpc-03cc1cc1e02d0160d",
        "ServiceName": "com.amazonaws.us-east-1.s3",
        "State": "available",
        "PolicyDocument": "{\"Version\":\"2008-10-17\",\"Statement\":[{\"Effect\":\"Allow\",\"Principal\":\"*\",\"Action\":\"*\",\"Resource\":\"*\"}]}",
        "RouteTableIds": [
            "rtb-0daedd7d2022f5ac6"
        ],
        "SubnetIds": [],
        "Groups": [],
        "PrivateDnsEnabled": false,
        "RequesterManaged": false,
        "NetworkInterfaceIds": [],
        "DnsEntries": [],
        "CreationTimestamp": "2023-03-16T04:29:59+00:00",
        "OwnerId": "066642259352"
    }
}

### Creating the Glue Service Role

a. Create an IAM Service role that would allow AWS Glue act in accessing S3 bucket and other resources:

In [ ]:
aws iam create-role --role-name my-glue-service-role --assume-role-policy-document '{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "glue.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}'

b. Grant Glue privileges on the S3 Bucket. 

Replacing the two blanks in the command below with an already-created S3 bucket.

The S3 path starts with `arn:aws:s3:::`. ARN is an AWS Resource Name, with the general format of `arn:[aws/aws-cn/aws-us-gov]:[service]:[region]:[account-id]:[resource-id]`.

The three points after S3 is because buckets can be cross-region or cross-account, for sharing data with client or versa. A cross account S3 bucket would be neccesary in such scenario:

In [ ]:
aws iam put-role-policy --role-name my-glue-service-role --policy-name S3Access --policy-document 
'{ "Version": "2012-10-17", 
    "Statement": [ 
        { 
            "Sid": "ListObjectsInBucket", 
            "Effect": "Allow", 
            "Action": [ "s3:ListBucket" ], 
            "Resource": [ "arn:aws:s3:::_______" ] 
        }, 
        { 
            "Sid": "AllObjectActions", 
            "Effect": "Allow", 
            "Action": "s3:*Object", 
            "Resource": [ "arn:aws:s3:::_______/*" ] 
        } 
    ] 
        }'

c. Give Glue Access to data in special S3 buckets used for Glue configurations

In [ ]:
aws iam put-role-policy --role-name my-glue-service-role --policy-name GlueAccess --policy-document '{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "glue:*",
                "s3:GetBucketLocation",
                "s3:ListBucket",
                "s3:ListAllMyBuckets",
                "s3:GetBucketAcl",
                "ec2:DescribeVpcEndpoints",
                "ec2:DescribeRouteTables",
                "ec2:CreateNetworkInterface",
                "ec2:DeleteNetworkInterface",
                "ec2:DescribeNetworkInterfaces",
                "ec2:DescribeSecurityGroups",
                "ec2:DescribeSubnets",
                "ec2:DescribeVpcAttribute",
                "iam:ListRolePolicies",
                "iam:GetRole",
                "iam:GetRolePolicy",
                "cloudwatch:PutMetricData"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:CreateBucket",
                "s3:PutBucketPublicAccessBlock"
            ],
            "Resource": [
                "arn:aws:s3:::aws-glue-*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:DeleteObject"
            ],
            "Resource": [
                "arn:aws:s3:::aws-glue-*/*",
                "arn:aws:s3:::*/*aws-glue-*/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::crawler-public*",
                "arn:aws:s3:::aws-glue-*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents",
                "logs:AssociateKmsKey"
            ],
            "Resource": [
                "arn:aws:logs:*:*:/aws-glue/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "ec2:CreateTags",
                "ec2:DeleteTags"
            ],
            "Condition": {
                "ForAllValues:StringEquals": {
                    "aws:TagKeys": [
                        "aws-glue-service-resource"
                    ]
                }
            },
            "Resource": [
                "arn:aws:ec2:*:*:network-interface/*",
                "arn:aws:ec2:*:*:security-group/*",
                "arn:aws:ec2:*:*:instance/*"
            ]
        }
    ]
}'

In [ ]:
'{ "Version": "2012-10-17", 
    "Statement": [ 
        { 
            "Sid": "ListObjectsInBucket", 
            "Effect": "Allow", 
            "Action": [ "s3:ListBucket" ], 
            "Resource": [ "arn:aws:s3:::semistify" ] 
        }, 
        { 
            "Sid": "AllObjectActions", 
            "Effect": "Allow", 
            "Action": "s3:*Object", 
            "Resource": [ "arn:aws:s3:::semistify/*" ] 
        } 
    ] 
        }'

In [ ]:
semistify